In [12]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
pd.set_option('display.max_columns', 200)

In [3]:
path = ""

In [55]:
data = pd.concat([pd.read_csv(path + 'Train.csv'), pd.read_csv(path + 'train_2.csv')], ignore_index=True)
data = data.drop_duplicates()
test = pd.read_csv(path + 'Test.csv')

In [56]:
test_columns = test.columns
ID = test.ID

In [57]:
data = data.drop(columns='ID')
data.shape

(30863, 4042)

In [58]:
data.shape, test.drop(columns='ID').shape

((30863, 4042), (13097, 4041))

In [59]:
combined = pd.concat([data, test], ignore_index=True).drop(columns='ID')

feature_cols = combined.columns.difference(['Accès internet'])

categorical_cols = combined[feature_cols].select_dtypes(include=['object', 'category']).columns
numeric_cols = combined[feature_cols].select_dtypes(include=['int64', 'float64']).columns

initial_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough' 
)

X_combined_transformed = initial_preprocessor.fit_transform(combined[feature_cols])

In [60]:
kmeans = KMeans(n_clusters=2, random_state=102)
kmeans_clusters = kmeans.fit_predict(X_combined_transformed)

combined['kmeans_cluster'] = kmeans_clusters

data = combined.iloc[:len(data)].copy()
test = combined.iloc[len(data):].copy().drop(columns='Accès internet').reset_index(drop=True)
test['ID'] = ID

In [61]:
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
numeric_cols = numeric_cols.drop('Accès internet')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), list(categorical_cols) +['kmeans_cluster'])
    ])

In [62]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=20, random_state=102))
])

In [64]:
model.fit(data.drop(columns='Accès internet'), data['Accès internet'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['TAILLE_MENAGE', ' ', ' .1', ' .2', ' .3', ' .4', ' .5', ' .6', ' .7',
       ' .8',
       ...
       ' .3990', ' .3991', ' .3992', ' .3993', ' .3994', ' .3995', ' .3996',
       ' .3997', ' .3998', ' .3999'],
      dtype='object', length=4001)),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['TypeLogmt_1', 'TypeLogmt_2',
                                                   'TypeLogmt_3', 'H08_Impute',
                                                   'H09_Impute', 'H17A', 'H17B',
                                                   'H17C', 'H17D', 'H17E',
                                                   'H17F', 'H17G', 'H17H',
                                                   'H17I', 'H17J', 'H17Y',
                                                   'H18A', 'H18B', 'H18C',
                                                   'H18D', 'H18E', 'H18F',
                                                   'H18G', 'H18H', 'H18I',
                                                   'H18J', 'H18Y', 'H20A',
                                                   'H20B', 'H20C', ...])])),
                ('classifier',
                 RandomForestClassifier(n_estimators=20, random_state=102))])

In [68]:
from sklearn.metrics import confusion_matrix
y_train_pred = model.predict(data.drop(columns='Accès internet'))
conf_matrix = confusion_matrix(data['Accès internet'], y_train_pred)

misclassified = (data['Accès internet'] != y_train_pred)
confusing_samples = data[misclassified]

clean_train = data[~misclassified]

In [71]:
clean_train.shape, data.shape

((30760, 4043), (30863, 4043))

In [72]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=102))
])

model.fit(clean_train.drop(columns='Accès internet'), clean_train['Accès internet'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['TAILLE_MENAGE', ' ', ' .1', ' .2', ' .3', ' .4', ' .5', ' .6', ' .7',
       ' .8',
       ...
       ' .3990', ' .3991', ' .3992', ' .3993', ' .3994', ' .3995', ' .3996',
       ' .3997', ' .3998', ' .3999'],
      dtype='object', length=4001)),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['TypeLogmt_1', 'TypeLogmt_2',
                                                   'TypeLogmt_3', 'H08_Impute',
                                                   'H09_Impute', 'H17A', 'H17B',
                                                   'H17C', 'H17D', 'H17E',
                                                   'H17F', 'H17G', 'H17H',
                                                   'H17I', 'H17J', 'H17Y',
                                                   'H18A', 'H18B', 'H18C',
                                                   'H18D', 'H18E', 'H18F',
                                                   'H18G', 'H18H', 'H18I',
                                                   'H18J', 'H18Y', 'H20A',
                                                   'H20B', 'H20C', ...])])),
                ('classifier', RandomForestClassifier(random_state=102))])

In [79]:
X_test_processed = model.named_steps['preprocessor'].transform(combined.iloc[len(data):].copy().drop(columns='Accès internet'))
test_predictions = model.named_steps['classifier'].predict_proba(X_test_processed)[:, 1]

submission = pd.DataFrame({'ID': ID, 'Target': test_predictions})
submission.to_csv('submission_220624_2.csv', index=False)